In [1]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

import sys
sys.path.insert(0, '../')
coralme_dir = '/home/chris/zuniga/coralme/'
sys.path.insert(0, coralme_dir)

import importlib
from cobra.core import Reaction, Metabolite
import coralme
import coralme.solver.solver
import coralme.builder.main
import coralme.core.model
import os
import pandas as pd
import json
import cobra
import numpy as np
import matplotlib.pyplot as plt
from coralme.util.helpers import get_nlp, optimize
import pickle

from coralme.builder.main import MEBuilder

# load ME and M models
base_dir = os.path.join(coralme_dir, 'species_files', 'Pseudomonas_files')
path = os.path.join(base_dir, 'individual_species', 'CP065866', 'outputs', 'MEModel-step3-Pseudomonas_aeruginosa_TJ2019-017-TS.pkl')
ME_model = coralme.io.pickle.load_pickle_me_model(path)


# Create a sink reaction
def add_sink_reaction_to_me(me_model, metabolite_id, lb = -.001, ub = -.001):
    """
    Add a sink reaction to a coralME model
    
    Parameters:
    -----------
    me_model : coralme.core.MEModel
        The ME model to add the sink to
    metabolite_id : str
        The ID of the metabolite to create a sink for
    lb : float
        Lower bound of the sink reaction
    ub : float
        Upper bound of the sink reaction
    """
    from coralme.core.reaction import MEReaction
    
    # Create the sink reaction
    sink_id = f'SINK_{metabolite_id}'
    sink_reaction = MEReaction(sink_id)
    sink_reaction.name = f'Sink reaction for {metabolite_id}'
    
    # Get the metabolite
    metabolite = me_model.metabolites.get_by_id(metabolite_id)
    
    # Add the metabolite to the reaction
    sink_reaction.add_metabolites({metabolite: -1})
    
    # Set the bounds
    sink_reaction.lower_bound = lb
    sink_reaction.upper_bound = ub
    
    # Add to model
    me_model.add_reactions([sink_reaction])
    
    return sink_reaction

In [ ]:
# Run the model
path = os.path.join(coralme_dir, 'data', 'metab_to_sol.pkl')
if os.path.exists(path):
    pickle_in = open(path, 'rb')
    metab_to_sol = pickle.load(pickle_in)
    pickle_in.close()
else:
    metab_to_sol = {}
    base_solution = ME_model.optimize(tolerance = 1e-3)
    metab_to_sol.update({'base' : base_solution})

for metab in ME_model.metabolites:
    if 'MONOMER' not in metab.id:
        continue
    if metab.id in metab_to_sol:
        continue

    # I want to create a sink reaction
    sink_reaction = add_sink_reaction_to_me(ME_model, metab.id)

    # Run the model, save results
    solution = ME_model.optimize(tolerance = 1e-3)
    metab_to_sol.update({metab.id : solution})
    pickle_out = open(path, 'wb')
    pickle.dump(metab_to_sol, pickle_out)
    pickle_out.close()

    # Remove the sink reaction when done
    ME_model.remove_reactions([sink_reaction])

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	1.4050280687025918	Not feasible
        2	0.7025140343512959	Not feasible
        3	0.3512570171756479	Not feasible
        4	0.1756285085878240	Not feasible
        5	0.0878142542939120	Not feasible
        6	0.0439071271469560	Optimal
        7	0.0658606907204340	Optimal
        8	0.0768374725071730	Not feasible
        9	0.0713490816138035	Not feasible
       10	0.0686048861671187	Optimal
       11	0.0699769838904611	Not feasible
       12	0.0692909350287899	Not feasible
Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	1.4050280687025918	Not feasible
        2	0.7025140343512959	Not feasible
        3	0.3512570171756479	Not feasible
        4	0.1756285085878240	Not feasible
        5	0.0878142542939120	Not feasible
        6	0.0439071271469560	Not feasible
        7	0.0219535635734780	Not feasible
        8	0.0109767817867390	Not feasible
   